In [1]:
import networkx as nx
import pandas as pd
import random
import torch
import pickle
from sklearn.model_selection import train_test_split
import numpy as np

In [3]:
unsw_labeled_path = "C:\\Users\\asus\\Documents\\nids-pcap-dataset\\unsw_parquet_used_dataset\\unsw_labeled.parquet"

In [4]:
### import parquet file
unsw_w_st = pd.read_parquet(unsw_labeled_path)

In [5]:
unsw_w_st.head(5)

,source_ip,destination_ip,source_port,destination_port,info_message,attack_category,is_malware,source_ip_info,source_port_info,dest_ip_info,dest_port_info,count_benign,count_malware
index,,,,,,,,,,,,,
1,175.45.176.1,149.171.126.18,4657,80,GET /oKmwKoVbq HTTP/1.1,NaN,0,175.45.176.1 GET /oKmwKoVbq HTTP/1.1,4657 GET /oKmwKoVbq HTTP/1.1,149.171.126.18 GET /oKmwKoVbq HTTP/1.1,80 GET /oKmwKoVbq HTTP/1.1,1,0
2,175.45.176.3,149.171.126.18,32473,80,GET /level/15/exec/-/buffers/assigned/dump HTT...,NaN,1,175.45.176.3 GET /level/15/exec/-/buffers/assi...,32473 GET /level/15/exec/-/buffers/assigned/du...,149.171.126.18 GET /level/15/exec/-/buffers/as...,80 GET /level/15/exec/-/buffers/assigned/dump ...,1,7
6,175.45.176.0,149.171.126.17,49194,80,GET eLWfxXSPkc HTTP/1.1,NaN,0,175.45.176.0 GET eLWfxXSPkc HTTP/1.1,49194 GET eLWfxXSPkc HTTP/1.1,149.171.126.17 GET eLWfxXSPkc HTTP/1.1,80 GET eLWfxXSPkc HTTP/1.1,1,0
9,175.45.176.1,149.171.126.14,51435,80,GET / HTTP/1.1,NaN,0,175.45.176.1 GET / HTTP/1.1,51435 GET / HTTP/1.1,149.171.126.14 GET / HTTP/1.1,80 GET / HTTP/1.1,46862,1313
10,175.45.176.1,149.171.126.19,64694,80,GET /scripts/cbag/ag.exe?page=FileDownload&id=...,NaN,0,175.45.176.1 GET /scripts/cbag/ag.exe?page=Fil...,64694 GET /scripts/cbag/ag.exe?page=FileDownlo...,149.171.126.19 GET /scripts/cbag/ag.exe?page=F...,80 GET /scripts/cbag/ag.exe?page=FileDownload&...,1,0


In [6]:
unsw_w_st.info()

<class 'pandas.core.frame.DataFrame'>
Index: 125180 entries, 1 to 490022
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   source_ip         125180 non-null  object  
 1   destination_ip    125180 non-null  object  
 2   source_port       125180 non-null  object  
 3   destination_port  125180 non-null  object  
 4   info_message      125180 non-null  object  
 5   attack_category   15657 non-null   category
 6   is_malware        125180 non-null  int64   
 7   source_ip_info    125180 non-null  object  
 8   source_port_info  125180 non-null  object  
 9   dest_ip_info      125180 non-null  object  
 10  dest_port_info    125180 non-null  object  
 11  count_benign      125180 non-null  int64   
 12  count_malware     125180 non-null  int64   
dtypes: category(1), int64(3), object(9)
memory usage: 12.5+ MB


In [7]:
unsw_w_st['source_port'] = unsw_w_st.source_port.astype('int32')
unsw_w_st['destination_port']= unsw_w_st.destination_port.astype('int32')

### Split Training Testing

In [22]:
train_df, test_df = train_test_split(unsw_w_st, test_size=0.3, random_state=0)

In [23]:
train_df.is_malware.value_counts()

is_malware
0    77142
1    10484
Name: count, dtype: int64

In [24]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87626 entries, 356428 to 256185
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   source_ip         87626 non-null  object  
 1   destination_ip    87626 non-null  object  
 2   source_port       87626 non-null  int32   
 3   destination_port  87626 non-null  int32   
 4   info_message      87626 non-null  object  
 5   attack_category   10995 non-null  category
 6   is_malware        87626 non-null  int64   
 7   source_ip_info    87626 non-null  object  
 8   source_port_info  87626 non-null  object  
 9   dest_ip_info      87626 non-null  object  
 10  dest_port_info    87626 non-null  object  
 11  count_benign      87626 non-null  int64   
 12  count_malware     87626 non-null  int64   
 13  sip_ohe           87626 non-null  int32   
dtypes: category(1), int32(3), int64(3), object(7)
memory usage: 8.4+ MB


In [25]:
value_counts = train_df.is_malware.value_counts()
if value_counts[1] > 160:
    df_to_lower = train_df[train_df['is_malware'] == 1].sample(n=160)
    train_df = pd.concat([train_df[train_df['is_malware'] == 0], df_to_lower])
    nD_train_df = train_df.copy()

In [48]:
nD_train_df = train_df.drop_duplicates(subset=['dest_port_info'])
label_train = nD_train_df['is_malware'].to_numpy()
label_train_tensor = torch.tensor(label_train, dtype=torch.float)
value_counts = np.unique(label_train, return_counts=True)
value_counts

(array([0, 1], dtype=int64), array([1623,  159], dtype=int64))

In [27]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37554 entries, 150048 to 143999
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   source_ip         37554 non-null  object  
 1   destination_ip    37554 non-null  object  
 2   source_port       37554 non-null  int32   
 3   destination_port  37554 non-null  int32   
 4   info_message      37554 non-null  object  
 5   attack_category   4662 non-null   category
 6   is_malware        37554 non-null  int64   
 7   source_ip_info    37554 non-null  object  
 8   source_port_info  37554 non-null  object  
 9   dest_ip_info      37554 non-null  object  
 10  dest_port_info    37554 non-null  object  
 11  count_benign      37554 non-null  int64   
 12  count_malware     37554 non-null  int64   
 13  sip_ohe           37554 non-null  int32   
dtypes: category(1), int32(3), int64(3), object(7)
memory usage: 3.6+ MB


In [28]:
nD_test_df = test_df.drop_duplicates(subset=['dest_port_info'])
nD_test_df.is_malware.value_counts()

is_malware
1    3044
0     849
Name: count, dtype: int64

In [30]:
value_counts = test_df.is_malware.value_counts()
if value_counts[1] > 849:
    df_to_lower = test_df[test_df['is_malware'] == 1].sample(n=849)
    test_df = pd.concat([test_df[test_df['is_malware'] == 0], df_to_lower])
    nD_test_df = test_df.copy()

In [49]:
nD_test_df = test_df.drop_duplicates(subset=['dest_port_info'])
label_test = nD_test_df['is_malware'].to_numpy()
value_counts = np.unique(label_test, return_counts=True)
label_test_tensor = torch.tensor(label_test, dtype=torch.float)
value_counts

(array([0, 1], dtype=int64), array([849, 757], dtype=int64))

In [32]:
test_df.is_malware.value_counts()

is_malware
0    33132
1      849
Name: count, dtype: int64

#### training graph

In [71]:
nD_graph_train = nx.Graph()
node_features = []
attr = []
labels = []

for dest_port_info in train_df["dest_port_info"].unique():
    nD_graph_train.add_node(dest_port_info)
    info_message = train_df[train_df["dest_port_info"] == dest_port_info]["info_message"].iloc[0]
    label = train_df[train_df["dest_port_info"] == dest_port_info]["is_malware"].iloc[0]
    node_features.append([float(len(info_message))])
    labels.append(label)
    
for (source_ip), group in train_df.groupby(["source_ip"]):
    for i in range(len(group) - 1):
        from_node = group.iloc[i]["dest_port_info"]
        to_node = group.iloc[i+1]["dest_port_info"]
        if nD_graph_train.has_edge(from_node, to_node):
            nD_graph_train[from_node][to_node]["weight"] += 1
        else:
            nD_graph_train.add_edge(from_node, to_node, weight=1)

In [72]:
label_train = torch.tensor(labels, dtype=torch.float)

In [73]:
node_features_tensor = torch.tensor(node_features)
node_features_tensor = node_features_tensor.cuda()

In [37]:
node_features_tensor

tensor([[ 28.],
        [199.],
        [ 15.],
        ...,
        [ 34.],
        [ 30.],
        [ 33.]], device='cuda:0')

In [38]:
## print num of nodes
print("Number of nodes:", nD_graph_train.number_of_nodes(), "\n")

Number of nodes: 1782 



In [39]:
isolated_nodes = list(nx.isolates(nD_graph_train))

print("Isolated nodes:", len(isolated_nodes), "\n")

Isolated nodes: 0 



#### testing graph

In [40]:
nD_graph_test = nx.Graph()
node_features_test = []
labels = []
ground_truth = []

for dest_port_info in test_df["dest_port_info"].unique():
    nD_graph_test.add_node(dest_port_info)
    info_message = test_df[test_df["dest_port_info"] == dest_port_info]["info_message"].iloc[0]
    label = test_df[test_df["dest_port_info"] == dest_port_info]["is_malware"].iloc[0]
    # print(info_message)
    node_features_test.append([float(len(info_message))])
    labels.append(label)

for (source_ip), group in test_df.groupby(["source_ip"]):
    for i in range(len(group) - 1):
        from_node = group.iloc[i]["dest_port_info"]
        to_node = group.iloc[i+1]["dest_port_info"]
        if nD_graph_test.has_edge(from_node, to_node):
            nD_graph_test[from_node][to_node]["weight"] += 1
        else:
            nD_graph_test.add_edge(from_node, to_node, weight=1)

In [41]:
node_features_tensor_test = torch.tensor(node_features_test)
node_features_tensor_test = node_features_tensor_test.cuda()

In [42]:
node_features_tensor_test

tensor([[28.],
        [15.],
        [24.],
        ...,
        [27.],
        [25.],
        [36.]], device='cuda:0')

In [43]:
isolated_nodes = list(nx.isolates(nD_graph_test))

print("Isolated nodes:", len(isolated_nodes), "\n")

Isolated nodes: 0 



### GAD

In [44]:
import torch
from torch_geometric.utils.convert import from_networkx
from pygod.detector import DOMINANT, OCGNN, GUIDE, GAE, GAAN, AnomalyDAE, CONAD
from pygod.metric import eval_average_precision, eval_roc_auc, eval_f1, eval_precision_at_k, eval_recall_at_k
from pygod.generator import gen_contextual_outlier, gen_structural_outlier
import pickle
device = torch.device('cuda')

In [74]:
pyG_train = from_networkx(nD_graph_train)
pyG_train = pyG_train
pyG_train.x = node_features_tensor

pyG_test = from_networkx(nD_graph_test)
pyG_test = pyG_test
pyG_test.x = node_features_tensor_test

#### DOMINANT

In [75]:
dominant_model = DOMINANT(gpu=0, weight=0.02, num_layers=144, hid_dim=64, contamination=0.1, lr=0.001, verbose=3, epoch=100)

In [76]:
dominant_compile = dominant_model.fit(pyG_train, label_train)

Epoch 0000: Loss 3.0377 | AUC 0.6184 | Recall 0.1132 | Precision 0.1132 | AP 0.1885 | F1 0.1132 | Time 0.92
Epoch 0001: Loss 3.0375 | AUC 0.6094 | Recall 0.1132 | Precision 0.1132 | AP 0.1879 | F1 0.1132 | Time 0.60
Epoch 0002: Loss 3.0372 | AUC 0.6093 | Recall 0.1132 | Precision 0.1132 | AP 0.1878 | F1 0.1132 | Time 0.58
Epoch 0003: Loss 3.0360 | AUC 0.6191 | Recall 0.1132 | Precision 0.1132 | AP 0.1896 | F1 0.1132 | Time 0.64
Epoch 0004: Loss 3.0315 | AUC 0.6190 | Recall 0.1132 | Precision 0.1132 | AP 0.1896 | F1 0.1132 | Time 0.58
Epoch 0005: Loss 3.0224 | AUC 0.6291 | Recall 0.1132 | Precision 0.1132 | AP 0.1917 | F1 0.1132 | Time 0.63
Epoch 0006: Loss 3.0166 | AUC 0.6561 | Recall 0.1195 | Precision 0.1195 | AP 0.1980 | F1 0.1195 | Time 0.52
Epoch 0007: Loss 3.0220 | AUC 0.6744 | Recall 0.1195 | Precision 0.1195 | AP 0.2030 | F1 0.1195 | Time 0.60
Epoch 0008: Loss 3.0133 | AUC 0.6654 | Recall 0.1195 | Precision 0.1195 | AP 0.2006 | F1 0.1195 | Time 0.61
Epoch 0009: Loss 3.0095 | AU

In [77]:
dominant_ip_pred_res, dominant_ip_score_res, dominant_ip_prob_res, dominant_ip_conf_res = dominant_compile.predict(data=pyG_test, label = label_test_tensor,return_pred=True, return_score=True, return_prob=True, prob_method='linear', return_conf=True)
unique_values, counts = torch.unique(dominant_ip_pred_res, return_counts=True)
print(unique_values, counts)

Test: Loss 0.0021 | AUC 0.7186 | Recall 0.6605 | Precision 0.6605 | AP 0.6132 | F1 0.6605 | Time 0.53
tensor([0, 1]) tensor([1411,  195])


In [87]:
f1_score_ip = eval_f1(label_test_tensor, dominant_ip_pred_res)
print(f1_score_ip)
precision = eval_precision_at_k(label_test_tensor, dominant_ip_score_res, k=1606)
print(precision)
recall = eval_recall_at_k(label_test_tensor, dominant_ip_score_res, k=1606)
print(recall)

0.18277310924369747
tensor(0.4714)
tensor(1.)


#### OCGNN

In [52]:
ocgnn_model = OCGNN(hid_dim=64, num_layers=64, weight_decay=0.02, 
                    contamination=0.1, lr=0.001, epoch=100, gpu=-1, batch_size=0, num_neigh=-1, 
                    beta=0.5, warmup=2, eps=0.001, verbose=3)

In [53]:
ocgnn_compile = ocgnn_model.fit(pyG_train, label_train_tensor)

Epoch 0000: Loss 0.0000 | AUC 0.9777 | Recall 0.7987 | Precision 0.7987 | AP 0.8374 | F1 0.7987 | Time 0.93
Epoch 0001: Loss 0.0002 | AUC 0.9665 | Recall 0.7987 | Precision 0.7987 | AP 0.7165 | F1 0.7987 | Time 0.77
Epoch 0002: Loss 0.0004 | AUC 0.9521 | Recall 0.7044 | Precision 0.7044 | AP 0.5467 | F1 0.7044 | Time 0.64
Epoch 0003: Loss 0.0002 | AUC 0.9501 | Recall 0.6918 | Precision 0.6918 | AP 0.5556 | F1 0.6877 | Time 0.68
Epoch 0004: Loss 0.0001 | AUC 0.2881 | Recall 0.0000 | Precision 0.0000 | AP 0.0740 | F1 0.0000 | Time 0.64
Epoch 0005: Loss 0.0001 | AUC 0.8981 | Recall 0.2327 | Precision 0.2327 | AP 0.3835 | F1 0.2327 | Time 0.67
Epoch 0006: Loss 0.0001 | AUC 0.9061 | Recall 0.2830 | Precision 0.2830 | AP 0.3988 | F1 0.2500 | Time 0.62
Epoch 0007: Loss 0.0001 | AUC 0.9433 | Recall 0.6478 | Precision 0.6478 | AP 0.5529 | F1 0.2776 | Time 0.78
Epoch 0008: Loss 0.0001 | AUC 0.9571 | Recall 0.7484 | Precision 0.7484 | AP 0.6556 | F1 0.7476 | Time 0.58
Epoch 0009: Loss 0.0001 | AU

In [88]:
ocgnn_ip_pred_res, ocgnn_ip_score_res, ocgnn_ip_prob_res, ocgnn_ip_conf_res = ocgnn_compile.predict(data=pyG_test, label=label_test_tensor, return_pred=True, return_score=True, return_prob=True, prob_method='linear', return_conf=True)

Test: Loss 0.0000 | AUC 0.1767 | Recall 0.2550 | Precision 0.2550 | AP 0.3196 | F1 0.2515 | Time 0.50


In [89]:
f1_score_ip = eval_f1(label_test_tensor, ocgnn_ip_pred_res)
print(f1_score_ip)
precision = eval_precision_at_k(label_test_tensor, ocgnn_ip_score_res, k=1606)
print(precision)
recall = eval_recall_at_k(label_test_tensor, ocgnn_ip_score_res, k=1606)
print(recall)

0.6179775280898877
tensor(0.4714)
tensor(1.)


#### GUIDE

In [57]:
guide_model = GUIDE(num_layers=4, hid_a=16, hid_s=16, weight_decay=0.05,  
                    contamination=0.1, lr=0.001, epoch=100, gpu=-1, dropout=0.5,
                    verbose=3)

In [58]:
print(guide_model)

GUIDE(act=<function relu at 0x0000027008028AF0>, alpha=0.5, backbone=None,
      batch_size=0, cache_dir=None, compile_model=False, contamination=0.1,
      dropout=0.5, epoch=100, gpu=None, graphlet_size=4, hid_a=None,
      hid_s=None, lr=0.001, num_layers=4, num_neigh=[-1, -1, -1, -1],
      save_emb=False, selected_motif=True, verbose=3, weight_decay=0.05)


In [59]:
guide_compile = guide_model.fit(pyG_train)

KeyboardInterrupt: 

In [ ]:
guide_ip_pred_res, guide_ip_score_res, guide_ip_prob_res, guide_ip_conf_res = guide_compile.predict(data=pyG_test, label=label_test_tensor, return_pred=True, return_score=True, return_prob=True, prob_method='linear', return_conf=True)
unique_values, counts = torch.unique(guide_ip_pred_res, return_counts=True)
print(unique_values, counts)

Test: Loss 0.8848 | AUC 1.0000 | Recall 1.0000 | Precision 1.0000 | AP 1.0000 | F1 1.0000 | Time 0.23
tensor([0, 1]) tensor([4440, 2583])


#### GAE

In [60]:
gae_model = GAE(hid_dim=64, num_layers=128, weight_decay=0.2, dropout=0.5,
                contamination=0.1, lr=0.001, epoch=100, gpu=-1,
                num_neigh=-1, verbose=3)

In [61]:
gae_compile = gae_model.fit(pyG_train, label_train_tensor)

Epoch 0000: Loss 91767.5156 | AUC 0.5823 | Recall 0.1384 | Precision 0.1384 | AP 0.2027 | F1 0.1373 | Time 2.16
Epoch 0001: Loss 91765.6406 | AUC 0.5753 | Recall 0.1321 | Precision 0.1321 | AP 0.2022 | F1 0.1321 | Time 1.30
Epoch 0002: Loss 91763.5391 | AUC 0.5759 | Recall 0.1384 | Precision 0.1384 | AP 0.2023 | F1 0.1384 | Time 1.49
Epoch 0003: Loss 91761.5703 | AUC 0.5730 | Recall 0.1321 | Precision 0.1321 | AP 0.2017 | F1 0.1321 | Time 1.10
Epoch 0004: Loss 91757.7578 | AUC 0.5765 | Recall 0.1321 | Precision 0.1321 | AP 0.2023 | F1 0.1321 | Time 1.07
Epoch 0005: Loss 91753.4375 | AUC 0.5763 | Recall 0.1321 | Precision 0.1321 | AP 0.2024 | F1 0.1321 | Time 1.06
Epoch 0006: Loss 91749.3281 | AUC 0.5790 | Recall 0.1321 | Precision 0.1321 | AP 0.2028 | F1 0.1321 | Time 1.06
Epoch 0007: Loss 91743.7109 | AUC 0.5762 | Recall 0.1321 | Precision 0.1321 | AP 0.2023 | F1 0.1321 | Time 1.12
Epoch 0008: Loss 91738.3984 | AUC 0.5774 | Recall 0.1384 | Precision 0.1384 | AP 0.2026 | F1 0.1384 | Ti

In [62]:
gae_ip_pred_res, gae_ip_score_res, gae_ip_prob_res, gae_ip_conf_res = gae_compile.predict(data=pyG_test, label=label_test_tensor, return_pred=True, return_score=True, return_prob=True, prob_method='linear', return_conf=True)
unique_values, counts = torch.unique(gae_ip_pred_res, return_counts=True)
print(unique_values, counts)

Test: Loss 110.0177 | AUC 0.6630 | Recall 0.6420 | Precision 0.6420 | AP 0.5840 | F1 0.6420 | Time 0.58
tensor([0, 1]) tensor([1336,  270])


#### GAAN

In [63]:
gaan_model = GAAN(noise_dim=8, hid_dim=8, num_layers=8, dropout=0.0, 
                  weight_decay=0.02, contamination=0.1, lr=0.004, 
                  epoch=100, gpu=-1, batch_size=0, num_neigh=0, 
                  weight=0.5, verbose=3)

In [89]:
gaan_compile = gaan_model.fit(pyG_train, label_train_tensor)

Epoch 0000: Loss G nan | Loss D nan | AUC 0.5810 | Recall 0.7866 | Precision 0.7866 | AP 0.8412 | F1 0.7866 | Time 2.25
Epoch 0001: Loss G nan | Loss D nan | AUC 0.5805 | Recall 0.7864 | Precision 0.7864 | AP 0.8410 | F1 0.7864 | Time 1.85
Epoch 0002: Loss G nan | Loss D nan | AUC 0.5805 | Recall 0.7858 | Precision 0.7858 | AP 0.8412 | F1 0.7858 | Time 1.86
Epoch 0003: Loss G nan | Loss D nan | AUC 0.5810 | Recall 0.7856 | Precision 0.7856 | AP 0.8413 | F1 0.7856 | Time 1.89
Epoch 0004: Loss G nan | Loss D nan | AUC 0.5808 | Recall 0.7859 | Precision 0.7859 | AP 0.8411 | F1 0.7859 | Time 1.93
Epoch 0005: Loss G nan | Loss D nan | AUC 0.5809 | Recall 0.7863 | Precision 0.7863 | AP 0.8411 | F1 0.7863 | Time 2.04
Epoch 0006: Loss G nan | Loss D nan | AUC 0.5804 | Recall 0.7866 | Precision 0.7866 | AP 0.8412 | F1 0.7866 | Time 1.98
Epoch 0007: Loss G nan | Loss D nan | AUC 0.5806 | Recall 0.7864 | Precision 0.7864 | AP 0.8413 | F1 0.7864 | Time 1.84
Epoch 0008: Loss G nan | Loss D nan | AU

In [90]:
gaan_ip_pred_res, gaan_ip_score_res, gaan_ip_prob_res, gaan_ip_conf_res = gaan_compile.predict(data=pyG_test, label = label_test_tensor,return_pred=True, return_score=True, return_prob=True, prob_method='linear', return_conf=True)
unique_values, counts = torch.unique(gaan_ip_pred_res, return_counts=True)
print(unique_values, counts)

Test: Loss nan | AUC 0.5099 | Recall 0.7938 | Precision 0.7938 | AP 0.8103 | F1 0.7938 | Time 0.48
tensor([0, 1]) tensor([3358,  487])


#### CONAD

In [64]:
conad_mode = CONAD(hid_dim=64, num_layers=64, dropout=0.0, weight_decay=0.0, contamination=0.1, lr=0.001, 
      epoch=100, gpu=-1, num_neigh=-1, weight=0.5, eta=0.5, margin=0.5, verbose=3)

In [65]:
conda_compile = conad_mode.fit(pyG_train, label_train_tensor)

Epoch 0000: Loss 20.2718 | AUC 0.5831 | Recall 0.1321 | Precision 0.1321 | AP 0.2030 | F1 0.1321 | Time 2.85
Epoch 0001: Loss 20.2727 | AUC 0.5742 | Recall 0.1321 | Precision 0.1321 | AP 0.2024 | F1 0.1321 | Time 1.89
Epoch 0002: Loss 20.2658 | AUC 0.5838 | Recall 0.1321 | Precision 0.1321 | AP 0.2037 | F1 0.1321 | Time 1.91
Epoch 0003: Loss 20.2611 | AUC 0.5839 | Recall 0.1321 | Precision 0.1321 | AP 0.2038 | F1 0.1321 | Time 1.78
Epoch 0004: Loss 20.2569 | AUC 0.5921 | Recall 0.1321 | Precision 0.1321 | AP 0.2052 | F1 0.1321 | Time 1.63
Epoch 0005: Loss 20.2510 | AUC 0.5855 | Recall 0.1321 | Precision 0.1321 | AP 0.2040 | F1 0.1321 | Time 1.64
Epoch 0006: Loss 20.2494 | AUC 0.5838 | Recall 0.1321 | Precision 0.1321 | AP 0.2036 | F1 0.1321 | Time 1.59
Epoch 0007: Loss 20.2440 | AUC 0.5838 | Recall 0.1321 | Precision 0.1321 | AP 0.2035 | F1 0.1321 | Time 1.38
Epoch 0008: Loss 20.2330 | AUC 0.5836 | Recall 0.1321 | Precision 0.1321 | AP 0.2033 | F1 0.1321 | Time 1.46
Epoch 0009: Loss 20

In [66]:
conad_ip_pred_res, conad_ip_score_res, conad_ip_prob_res, conad_ip_conf_res = conda_compile.predict(data=pyG_test, label = label_test_tensor,return_pred=True, return_score=True, return_prob=True, prob_method='linear', return_conf=True)
unique_values, counts = torch.unique(conad_ip_pred_res, return_counts=True)
print(unique_values, counts)

Test: Loss 0.0306 | AUC 0.6676 | Recall 0.6420 | Precision 0.6420 | AP 0.5881 | F1 0.6420 | Time 0.70
tensor([0, 1]) tensor([1344,  262])


#### AnomalyDAE

In [68]:
anomalydae_model = AnomalyDAE(emb_dim=128, hid_dim=128, num_layers=68, weight_decay=0.02, 
                            alpha=0.5, theta=1.0, eta=1.0, 
                            contamination=0.1, lr=0.004, epoch=100, gpu=-1, 
                            verbose=3)

In [69]:
anomalydae_compile = anomalydae_model.fit(pyG_train, label_train_tensor)

Epoch 0000: Loss 4816.1001 | AUC 0.7670 | Recall 0.3270 | Precision 0.3270 | AP 0.3926 | F1 0.3291 | Time 0.41
Epoch 0001: Loss 520817.8750 | AUC 0.7674 | Recall 0.3333 | Precision 0.3333 | AP 0.3945 | F1 0.3333 | Time 0.21
Epoch 0002: Loss 211217.8438 | AUC 0.7667 | Recall 0.3333 | Precision 0.3333 | AP 0.3947 | F1 0.3376 | Time 0.35
Epoch 0003: Loss 28883.8184 | AUC 0.7662 | Recall 0.3333 | Precision 0.3333 | AP 0.3949 | F1 0.3333 | Time 0.19
Epoch 0004: Loss 7822.3154 | AUC 0.7756 | Recall 0.3333 | Precision 0.3333 | AP 0.3976 | F1 0.3333 | Time 0.18
Epoch 0005: Loss 18032.4980 | AUC 0.7979 | Recall 0.3082 | Precision 0.3082 | AP 0.2699 | F1 0.3082 | Time 0.16
Epoch 0006: Loss 14277.4443 | AUC 0.8430 | Recall 0.2893 | Precision 0.2893 | AP 0.3271 | F1 0.2893 | Time 0.16
Epoch 0007: Loss 5742.2998 | AUC 0.8481 | Recall 0.3019 | Precision 0.3019 | AP 0.3396 | F1 0.3019 | Time 0.16
Epoch 0008: Loss 18290.9297 | AUC 0.8689 | Recall 0.3396 | Precision 0.3396 | AP 0.3755 | F1 0.3396 | Tim

In [70]:
anomalydae_ip_pred_res, anomalydae_ip_score_res, anomalydae_ip_prob_res, anomalydae_ip_conf_res = conda_compile.predict(data=pyG_test, label = label_test_tensor,return_pred=True, return_score=True, return_prob=True, prob_method='linear', return_conf=True)

Test: Loss 0.0306 | AUC 0.6676 | Recall 0.6420 | Precision 0.6420 | AP 0.5881 | F1 0.6420 | Time 0.33
tensor([0, 1]) tensor([1344,  262])
